In [1]:
import numpy as np
from PIL import Image, ImageDraw
import os
import random

### Gestalt Principle: CLOSURE

For this problem, I have chosen the Gestalt principle of closure. The Gestalt principle of closure suggests that our brains tend to perceive incomplete shapes, patterns, or forms as complete or whole.

!['incomplete circle'](closure_dataset/val_dataset/circle_unclosed_218.png)
!['complete circle'](closure_dataset/train_dataset/circle_closed_11.png)

I will try to train a CNN model which can recognize whole shapes even if they are incomplete. If it is sufficiently successful in recognizing incomplete images as whole, then I will be able to say with some level of assurance that the model was able to learn the principle of closure.

### Generating datasets

I have generated images of two figures: **Cicles** and **Squares** with various levels of (in)completeness across the training, validations and testing datasets. The testing datast contains a **higher** proportion of incomplete images to test for the CLOSURE property. Certain transformations like rotation and translation have been applied to make the dataset more generalized.

There are 1000 sample images for each of train, validation and test dataset.

In [75]:
def create_closed_shape(draw, shape_type, coords):
    if shape_type == 'circle':
        draw.arc(coords, start=0, end=360, fill='black')
        return "C"
    elif shape_type == 'square':
        draw.rectangle(coords, outline='black')
        return "S"
    elif shape_type == 'triangle':
        draw.polygon(coords, outline='black')
        return "T"

In [76]:
# Draw an unclosed shape based on its type and completeness
def create_unclosed_shape(draw, shape_type, coords, completeness):
    if shape_type == 'circle':
        # Only draw the portion of the circle corresponding to completeness
        angle = int(360 * completeness)  # Compute the angle of the arc to draw
        draw.arc(coords, start=0, end=angle, fill='black')
        return "C"
        
    elif shape_type == 'square':
        # Break the square perimeter into segments (4 equal sides)
        total_length = 4  # Square has 4 sides
        sides_to_draw = int(total_length * completeness)
        side_fraction = completeness - sides_to_draw / total_length  # Remaining portion of the last side
        
        # Draw the sides based on completeness
        if sides_to_draw >= 1:
            draw.line([coords[0], coords[1], coords[2], coords[1]], fill='black')  # Top line
        if sides_to_draw >= 2:
            draw.line([coords[2], coords[1], coords[2], coords[3]], fill='black')  # Right line
        if sides_to_draw >= 3:
            draw.line([coords[2], coords[3], coords[0], coords[3]], fill='black')  # Bottom line
        if sides_to_draw == 4:
            draw.line([coords[0], coords[3], coords[0], coords[1]], fill='black')  # Left line

        # Partial side drawing based on the side fraction
        if side_fraction > 0:
            if sides_to_draw == 0:
                draw.line([coords[0], coords[1], coords[0] + (coords[2] - coords[0]) * side_fraction, coords[1]], fill='black')  # Partial top line
            elif sides_to_draw == 1:
                draw.line([coords[2], coords[1], coords[2], coords[1] + (coords[3] - coords[1]) * side_fraction], fill='black')  # Partial right line
            elif sides_to_draw == 2:
                draw.line([coords[2], coords[3], coords[2] - (coords[2] - coords[0]) * side_fraction, coords[3]], fill='black')  # Partial bottom line
            elif sides_to_draw == 3:
                draw.line([coords[0], coords[3], coords[0], coords[3] - (coords[3] - coords[1]) * side_fraction], fill='black')  # Partial left line
        return "S"

    elif shape_type == 'triangle':
        # Break the triangle perimeter into segments (3 sides)
        total_segments = 3
        segments_to_draw = int(total_segments * completeness)
        segment_fraction = completeness - segments_to_draw / total_segments
        
        # Draw full sides
        if segments_to_draw >= 1:
            draw.line([coords[0], coords[1], coords[2], coords[3]], fill='black')  # Base
        if segments_to_draw >= 2:
            draw.line([coords[2], coords[3], coords[4], coords[5]], fill='black')  # Left slope
        if segments_to_draw == 3:
            draw.line([coords[4], coords[5], coords[0], coords[1]], fill='black')  # Right slope
        
        # Partial side drawing based on the segment fraction
        if segment_fraction > 0:
            if segments_to_draw == 0:
                draw.line([coords[0], coords[1], coords[0] + (coords[2] - coords[0]) * segment_fraction, coords[1] + (coords[3] - coords[1]) * segment_fraction], fill='black')  # Partial base line
            elif segments_to_draw == 1:
                draw.line([coords[2], coords[3], coords[2] + (coords[4] - coords[2]) * segment_fraction, coords[3] + (coords[5] - coords[3]) * segment_fraction], fill='black')  # Partial left slope
            elif segments_to_draw == 2:
                draw.line([coords[4], coords[5], coords[0] + (coords[0] - coords[4]) * segment_fraction, coords[1] + (coords[1] - coords[5]) * segment_fraction], fill='black')  # Partial right slope
        return "T"

In [74]:
def apply_random_transformations(img):
    """Apply rough transformations to the image."""
    # Random rotation
    if random.random() < 0.5:  # Apply transformation with 50% probability
        angle = random.randint(-30, 30)  # Rotate between -30 and +30 degrees
        img = img.rotate(angle, expand=True)
        
    # Random translation
    if random.random() < 0.5:  # Apply transformation with 50% probability
        translate_x = random.randint(-30, 30)  # Larger translation
        translate_y = random.randint(-30, 30)
        img = img.transform(img.size, Image.AFFINE, (1, 0, translate_x, 0, 1, translate_y), fillcolor='white')

    return img

In [78]:
# Create an image with a shape, either closed or unclosed
def create_image(shape_type, img_size=(200, 200), is_closed=True, completeness=1.0):
    img = Image.new('RGB', img_size, 'white')
    draw = ImageDraw.Draw(img)
    
    # Shape coordinates based on type
    if shape_type == 'circle':
        coords = [50, 50, 150, 150]
    elif shape_type == 'square':
        coords = [50, 50, 150, 150]  # Square coordinates
    elif shape_type == 'triangle':
        coords = [50, 150, 150, 150, 100, 50]  # Triangle coordinates
    
    # Draw the shape based on whether it's closed or unclosed
    if is_closed:
        shape_name = create_closed_shape(draw, shape_type, coords)
    else:
        shape_name = create_unclosed_shape(draw, shape_type, coords, completeness)
    
    #  Apply random rough transformations
    img = apply_random_transformations(img)
    
    return img, shape_name

In [90]:
# Generate a dataset of images
def generate_dataset(dataset_type, img_size=(200, 200), num_images=100, closed_ratio=0.5, completeness= 0.5):
    base_path = f"./closure_dataset/{dataset_type}_dataset/"
    os.makedirs(base_path, exist_ok=True)
    shape_types = ['circle', 'square']
    labels = []
    for i in range(num_images):
        shape_type = np.random.choice(shape_types)
        is_closed = np.random.rand() < closed_ratio

        save_path = base_path + f"{shape_type}_{'closed' if is_closed else 'unclosed'}_{i}.png"
        img, shape_name = create_image(shape_type, img_size=img_size, is_closed=is_closed, completeness=completeness)
        labels.append(f"{shape_name}")
        img.save(save_path)
    
    return labels

In [91]:
# Generate 1000 training images
train_labels = generate_dataset('train', num_images=1000, img_size=(200, 200), closed_ratio=0.8, completeness=0.75)
# Generate 1000 validation images
val_labels = generate_dataset('val', num_images=1000, img_size=(200, 200), closed_ratio=0.8, completeness=0.5)
# Generate 1000 testing images
test_labels = generate_dataset('test', num_images=1000, img_size=(200, 200), closed_ratio=0.1, completeness=0.3)